In [1]:
import nltk
import numpy
import pandas
import sklearn

import sqlite3

import csshelper
import cssfeature
import csspipe
import csstransformer


## Load data
(run bash ./src/init.sh first!)

In [3]:
#data = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
#help(data.head)
#data.head(1)
db_path = './data.sqlite3'
sqlite_connection = sqlite3.connect(db_path)

cNeu_reader = csshelper.CNeuReader(sqlite_connection)
sNeu_reader = csshelper.SNeuReader(sqlite_connection)

print(cNeu_reader.get_results().head(3))
print(sNeu_reader.get_results().head(3))



                                            sentence  cNeu
0                        likes the sound of thunder.  True
1  is so sleepy it's not even funny that's she ca...  True
2  is sore and wants the knot of muscles at the b...  True
                                            sentence  sNeu
0                        likes the sound of thunder.     3
1  is so sleepy it's not even funny that's she ca...     3
2  is sore and wants the knot of muscles at the b...     3


# Create DataFrame with string-length-featues

In [9]:

print(data.head(2))

## create length of sentence feature
#sl_data = cssfeature.CharacterFeatures.string_length(cNeu_reader.get_results(), column=0, column_name='sentence_length')

#print(sl_data.head(2))


                                            sentence  cNeu
0                        likes the sound of thunder.  True
1  is so sleepy it's not even funny that's she ca...  True


In [10]:
features = sl_data[['sentence_length']].astype(str)
labels = sl_data[['cNeu']].astype(bool)

print(features.head(2))
print(labels.head(2))

NameError: name 'sl_data' is not defined

In [11]:

split = sklearn.cross_validation.train_test_split(features, labels, train_size = 0.66, random_state= 6432119)

train_feat, test_feat, train_label, test_label = split

"""
print("train_feat\n%s\n" %train_feat.head(1))
print("train_label\n%s\n" %train_label.head(1))
print("test_feat\n%s\n" %test_feat.head(1))
print("test_label\n%s\n" %test_label.head(1))
"""


NameError: name 'features' is not defined

# Create pipefactory

In [12]:
pipefactory = csspipe.PipeFactory()

### Create classifiers for testing

In [13]:

multinomial_nb = pipefactory.classifiers.multinomial_nb
bernoulli_nb = pipefactory.classifiers.bernoulli_nb

svc = pipefactory.classifiers.svc
linear_svc = pipefactory.classifiers.linear_svc

random_forest = pipefactory.classifiers.random_forest_classifier
k_neighbors = pipefactory.classifiers.k_neighors_classifier


## Create extractors for testing

In [14]:

tfidf_vectorizer = pipefactory.extractors.tfidf_vectorizer


## Create selectors for testing

In [15]:
k_best = pipefactory.selectors.select_k_best

In [16]:
import nltk
from nltk import tag
from nltk.corpus import treebank

postagger = nltk.tag.ClassifierBasedPOSTagger(train=treebank.tagged_sents())



In [17]:
mytagger = csstransformer.PartOfSpeech(ignore_unknown=True)

In [18]:
#postagger.evaluate(treebank.tagged_sents())
input = data.sentence[:5,]
print()
#print(postagger.tag_sents(input))
print()
#print(mytagger.transform(input))

## Findings:
-  ngrams ranging from 2-3 return better results than 1-1
- full Part-Of-Speech of a sentence + tfidf does not behave well

###Why pos tagging failed:
We used tfidf to build vectors. tfidf's purpose however, is to *minimize the impact of frequently repeting components* (words). Thus the "part of speech"-equivalent of a sentence consisting of a description such as 'NN' will have less impact than a sentence build out of real words, as they are way less frequent in the whole test data.


###POS-Workarounds
- instead of converting a full sentence into POS-Tag one could try to filter all nouns or adjectives


In [26]:
## Pipelineing
# http://scikit-learn.org/stable/modules/pipeline.html
# http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html

class Test(object):
    
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, input):
        return postagger.tag_sents(input)



raw_pipeline = pipefactory.create_pipe([
        #tfidf_vectorizer
        ('tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(2,3), analyzer='word'))
        #, ('kNN', sklearn.neighbors.KNeighborsClassifier())
        , multinomial_nb
    ])

pos_pipeline = pipefactory.create_pipe([
        #tfidf_vectorizer
        #('part of speech', Test())
        ('part of speech', csstransformer.PartOfSpeech(ignore_unknown=True))
        #('part of speech', csstransformer.NounTransformer())
        #, ('count vectorizer', sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1)))
        , ('tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(2,3), analyzer='word'))
        , ('k_best', sklearn.feature_selection.SelectKBest(sklearn.feature_selection.chi2, k=1000))
        #, ('kNN', sklearn.neighbors.KNeighborsClassifier())
        , multinomial_nb
    ])



data = csshelper.SNeuReader(sqlite_connection).get_results()


print(data)

r1 = sklearn.cross_validation.cross_val_score(raw_pipeline, data.sentence.astype(str), data.sNeu.astype(bool), cv=5)
r2 = sklearn.cross_validation.cross_val_score(pos_pipeline, data.sentence.astype(str), data.sNeu.astype(bool), cv=5)

print(r1.mean())
print(r2.mean())

print(pos_pipeline)


                                               sentence  sNeu
0                           likes the sound of thunder.     3
1     is so sleepy it's not even funny that's she ca...     3
2     is sore and wants the knot of muscles at the b...     3
3            likes how the day sounds in this new song.     3
4                                           is home. <3     3
5                                 www.thejokerblogs.com     3
6     saw a nun zombie, and liked it. Also, *PROPNAM...     3
7     is in Kentucky. 421 miles into her 1100 mile j...     3
8     was about to finish a digital painting before ...     3
9     is celebrating her new haircut by listening to...     3
10                    has a crush on the Green Lantern.     3
11                              has magic on the brain.     3
12    saw Transformers, Up, and Year One this week. ...     3
13    Who wants to meet up on schedule pick-up day a...     3
14      desires the thrill of inspiration. Also, money.     3
15      

In [36]:
test_data = data['sentence'].astype(str).values[:3]

print("test_data:\n%s" % test_data)

tfidf_trans = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,2))
#help(tfidf_trans)
tfidf_trans.fit(test_data)
#tfidf_trans.transform(test_data)

print(tfidf_trans.get_feature_names()[:10])

type(tfidf_trans.transform(data))
print(tfidf_trans.transform(["funny", "and"]).toarray())

#tfidf_trans.get_feature_names()

test_data:
['likes the sound of thunder.'
 "is so sleepy it's not even funny that's she can't get to sleep."
 "is sore and wants the knot of muscles at the base of her neck to stop hurting. On the other hand, YAY I'M IN ILLINOIS! <3"]
['and', 'and wants', 'at', 'at the', 'base', 'base of', 'can', 'can get', 'even', 'even funny']
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]]


In [25]:
tfidf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer()

pipeline = sklearn.pipeline.Pipeline([('tfidf', tfidf_vectorizer),
                     ('k_best', sklearn.feature_selection.SelectKBest(sklearn.feature_selection.chi2, k=1000)),
                     ('nb', sklearn.naive_bayes.MultinomialNB())
                    ])

#pipeline.fit(train_feat[["STATUS"]], train_label[["cNEU"]])
#data = train_feat.STATUS
#target = train_label.cNEU.apply(lambda x: x == 'y')

#foo = tfidf_vectorizer.fit_transform(data[:1], target[:1])
#print(foo)
#idf = foo.idf_

#print(len(idf))
#print(len(data))

#pipeline.fit(data.sentence, data.cNeu)
import sklearn
import sklearn.feature_extraction.text

data = csshelper.CNeuReader(sqlite_connection).get_results()

features = data.sentence
labels = data.cNeu

cv = sklearn.feature_extraction.text.CountVectorizer()

cv_fit = cv.fit(features, labels)
cv_transform_features = cv_fit.transform(features)


tfidftrans = sklearn.feature_extraction.text.TfidfTransformer()

tfidftrans_fit = tfidftrans.fit(cv_transform_features, labels)
tfidftrans_transform_features = tfidftrans_fit.transform(cv_transform_features)

#print(cv_transform_features)
#print(tfidftrans_transform_features)

skb = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.chi2, k=1000)
mnb = sklearn.naive_bayes.MultinomialNB()



mnb_fit = mnb.fit(tfidftrans_transform_features, labels.astype(bool))

print(hasattr(mnb_fit, "fit"))
print(hasattr(mnb_fit, "transform"))
print(hasattr(mnb_fit, "fit_transform"))

pipe = sklearn.pipeline.Pipeline([
    ('cv', sklearn.feature_extraction.text.CountVectorizer())
    , ('tfidf_trans', sklearn.feature_extraction.text.TfidfTransformer())
    , ('nb', sklearn.naive_bayes.MultinomialNB())
])

sklearn.cross_validation.cross_val_score(pipe, features.astype(str), labels.astype(bool))



#print(tfidf_vectorizer.fit_transform(features, labels))
#tfidf = tfidf_vectorizer.fit_transform(features, labels)
#print(tfidf)

#print(tfidf[[9]])


select_k = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.chi2, k=1000)
print('select_k has transform: %s' % hasattr(select_k, 'transform'))
print('select_k has fit: %s' % hasattr(select_k, 'fit'))
print('select_k has fit_transform: %s' % hasattr(select_k, 'fit_transform'))


#help(tfidf_vectorizer)

#result = sk.cross_validation.cross_val_score(pipeline, data[[0]], data[[1]])

#print(result)

True
False
False
select_k has transform: True
select_k has fit: True
select_k has fit_transform: True


In [26]:
pipe_factory = csspipe.PipeFactory()


In [27]:
tfidf_nb_pipe = pipe_factory.create_tfidf_nb()

In [28]:
tfidf_nb_pipe.steps[0]

('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words=None, strip_accents=None, sublinear_tf=False,
         token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
         vocabulary=None))

In [30]:
result = sklearn.cross_validation.cross_val_score(tfidf_nb_pipe, data[[0]].values, data[[1]].values)
print(result)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'